Course: https://www.youtube.com/playlist?list=PLNKXA-74YGLh9YbfTm2x_VUYtPXVQ5Zsw.

Implementation of LU decomposition with partial pivoting
from "scratch".

In [215]:
import numpy as np

In [ ]:
A = np.array([ #n*n sample matrix
    [0, 1, 1],
    [3, 2, 0],
    [2, 0, 3],
])
print(A.shape)
print(A)

(3, 3)
[[0 1 1]
 [3 2 0]
 [2 0 3]]


In [ ]:
P = np.zeros(A.shape)

perm = np.arange(A.shape[0]) #Pivoting (greedy), not optimal
l = 0
while(l < len(perm)): #It's better to make this procedure together with calculations of L and U
    mx, pos = 0, 0
    for i in range(l, len(perm)):
        val = abs(A[perm[i]][l])
        if val > mx:
            mx = val
            pos = i
    perm[l], perm[pos] = perm[pos], perm[l] #Greatest element on diagonal
    l += 1

print(perm)
for i in enumerate(perm): #Pivot matrix
    P[i] = 1
A = P @ A
print(A)

[1 0 2]
[[3. 2. 0.]
 [0. 1. 1.]
 [2. 0. 3.]]


In [218]:
L, U = np.zeros(A.shape), np.zeros(A.shape)
A_temp = A.copy()

eps = 1e-5
for i in range(A.shape[0]):
    L[i, i] = 1
    U[i, i:] = A_temp[i, i:] #U - backward subst (transformation matrix to get to REF)
    L[i+1:, i] = A_temp[i+1:, i] / U[i, i] #L - Forward subst (Actual REF)

    if abs(U[i, i]) < eps:
        raise ValueError("Pivot is zero!")

    A_temp[i+1:, i+1:] -= np.outer(L[i+1:, i], U[i, i+1:]) #Step

print(L, U, sep="\n\n")

[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.66666667 -1.33333333  1.        ]]

[[3.         2.         0.        ]
 [0.         1.         1.        ]
 [0.         0.         4.33333333]]


In [ ]:
def forward_substitution(L, b): #LUx = b (Coefficients of REF)
    y = np.zeros(b.shape)
    n = L.shape[0]
    for i in range(n):
        y[i] = b[i] - L[i, :i] @ y[:i] #Finding coefficients
    return y

def backward_substitution(U, y): #Ly = b (Ux = y) (RREF)
    x = np.zeros(y.shape)
    n = U.shape[0]
    for i in np.arange(n - 1, -1, -1):
        x[i] = (y[i] - U[i, i+1:] @ x[i+1:]) / U[i, i] #Finding solutions
    return x

b = np.array([1, 2, 3])

def solve(L, U, b): #Solving for x
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    return x

#Small test
x = solve(L, U, b)
print("Ax = ", A @ x)
print("b  =", b)
print("SOL OK:", np.allclose(A @ x, b))
print("LU OK:", np.allclose(A, L @ U))
#O(N^3) for the first equation, then O(N^2) because we've already calculated L, U

Ax =  [1. 2. 3.]
b  = [1 2 3]
SOL OK: True
LU OK: True
